# Preprocess survey data

## Load and clean data

In [ ]:
!pip install cite

# Drive
from google.colab import drive

# Util
import os
import re
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 37.2 MB/s 


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "file path here"

In [ ]:
# Clean dataframe
def clean_df(df):
  # Remove irrelevant columns
  df.drop(df.columns[0:19], axis=1, inplace=True)
  df.drop(df.columns[92:95], axis=1, inplace=True)
  df.drop([0,1], axis=0, inplace=True)
  df.reset_index(drop=True, inplace=True)

  # Rename one question that for some reason has a weird name
  df.rename({"Q97_1":"QD-R9_1", "Q97_2":"QD-R9_2", "Q97_3":"QD-R9_3"}, axis=1, inplace=True)

  for col in df.columns:
    for i in range(len(df)):
      if df[col].iloc[i] is np.nan:
        df[col].iloc[i] = 0 # Set all NaN's to zero
      else:
        df[col].iloc[i] = int(df[col].iloc[i])    
      
  return df

In [ ]:
# Load survey data
df_all = pd.read_csv(file_path+"survey_all.csv")

df_all = clean_df(df_all)

0


In [ ]:
df_filt = pd.read_csv(file_path+"survey_filtered.csv") # Filtered for EN>=2, DE=3


df_filt = clean_df(df_filt)

In [ ]:
# for i in range(len(df_all.columns)):
#   print(i)
#   print(df_all[df_all.columns[i]].iloc[0])

## FUNC: Fixed questions

In [ ]:
# df for all fixed questions
def fixed_q(df):
  fix =  df.copy(deep=True)
  fix.drop(df.columns[4:28], axis=1, inplace=True)
  fix.drop(df.columns[32:56], axis=1, inplace=True)
  fix.drop(df.columns[62:92], axis=1, inplace=True)
  return fix

# df for sections B and C
def fixed_q_BC(df):
  fix =  df.copy(deep=True)
  fix.drop(df.columns[4:28], axis=1, inplace=True)
  fix.drop(df.columns[32:92], axis=1, inplace=True)
  return fix

# df for section D
def fixed_q_D(df):
  fix =  df.copy(deep=True)
  fix.drop(df.columns[0:56], axis=1, inplace=True)
  fix.drop(df.columns[62:92], axis=1, inplace=True)
  return fix

## FUNC: Group or Relabel Categories (to avoid overlap)


In [ ]:
# OPTIONAL, might improve interrater agreement
# Group first 2 parts into 4 categories ([don't know]=0, [incomprehensible, bad]=1, [adequate]=2,[good, flawless]=3)
def group_categories_overlap(df):
  grouped_df = df.copy(deep=True)
  groupdict = {0:0, 1:1, 2:1, 3:2, 4:3, 5:3}
  for col in grouped_df.columns[0:8]:
    for i in range(len(grouped_df)):
      grouped_df[col].iloc[i] = groupdict[grouped_df[col].iloc[i]]
  return grouped_df

In [ ]:
# ENSURES FOR NO OVERLAP BETWEEN B-C and D

# OPTIONAL, might improve interrater agreement
# Group first 2 parts into 4 categories ([don't know]=0, [incomprehensible, bad]=4, [adequate]=5,[good, flawless]=6)
def group_categories(df):
  grouped_df = df.copy(deep=True)
  # changed it to [4,5,6] instead of [1,2,3] to avoid overlap with Comparison group
  groupdict = {0:0, 1:4, 2:4, 3:5, 4:6, 5:6}
  for col in grouped_df.columns[0:8]:
    for i in range(len(grouped_df)):
      grouped_df[col].iloc[i] = groupdict[grouped_df[col].iloc[i]]
  return grouped_df

In [ ]:
# Simple relabelling!
def relabel_categories(df):
  grouped_df = df.copy(deep=True)
  # changed it to [4,5,6,7,8] instead of [1,2,3,4,5] to avoid overlap with Comparison group
  groupdict = {0:0, 1:4, 2:5, 3:6, 4:7, 5:8}
  for col in grouped_df.columns[0:8]:
    for i in range(len(grouped_df)):
      grouped_df[col].iloc[i] = groupdict[grouped_df[col].iloc[i]]
  return grouped_df

In [ ]:
fix = fixed_q(df_filt)

## Step 1 -  Isolate B & C from D

Isolate B and C from D

ALL

In [ ]:
fixD = fixed_q_D(df_all)

In [ ]:
fixBC = fixed_q_BC(df_all)

FILTERED

In [ ]:
fixD = fixed_q_D(df_filt)

In [ ]:
fixBC = fixed_q_BC(df_filt)

In [ ]:
fixBC

,QB-F1-1,QB-F1-2,QB-F2-1,QB-F2-2,QC-F1-1,QC-F1-2,QC-F2-1,QC-F2-2
0,4,3,5,4,5,5,3,2
1,2,3,1,1,1,3,2,2
2,4,3,4,4,5,4,4,2
3,3,2,3,1,2,3,2,2
4,5,3,5,5,3,3,2,1
5,4,2,5,5,5,5,4,2
6,4,5,4,2,2,2,3,4
7,3,2,4,4,3,2,3,2
8,4,2,5,4,2,4,1,1
9,2,3,3,3,2,5,2,2


## Step 2 (A) -  Relabel (1,2,3,4,5) -> (4,5,6,7,8)


Relabel instead of Group

In [ ]:
fixBC_relabel = relabel_categories(fixBC) 
fixBC_relabel

,QB-F1-1,QB-F1-2,QB-F2-1,QB-F2-2,QC-F1-1,QC-F1-2,QC-F2-1,QC-F2-2
0,7,6,8,7,8,8,6,5
1,5,6,4,4,4,6,5,5
2,7,6,7,7,8,7,7,5
3,6,5,6,4,5,6,5,5
4,8,6,8,8,6,6,5,4
5,7,5,8,8,8,8,7,5
6,7,8,7,5,5,5,6,7
7,6,5,7,7,6,5,6,5
8,7,5,8,7,5,7,4,4
9,5,6,6,6,5,8,5,5


Recombine fixBC and fixD - RELABELLED

In [ ]:
fixBC_relabel.index.name = "Index"
fixD.index.name = "Index"

In [ ]:
fix_relabel = fixBC_relabel.merge(fixD, on="Index")

df = fix_relabel

df

,QB-F1-1,QB-F1-2,QB-F2-1,QB-F2-2,QC-F1-1,QC-F1-2,QC-F2-1,QC-F2-2,QD-F1_1,QD-F1_2,QD-F1_3,QD-F2_1,QD-F2_2,QD-F2_3
Index,,,,,,,,,,,,,,
0,7,6,8,7,8,8,6,5,1,2,3,2,3,1
1,5,6,4,4,4,6,5,5,2,1,3,2,3,1
2,7,6,7,7,8,7,7,5,2,3,1,2,1,3
3,6,5,6,4,5,6,5,5,1,2,3,1,3,2
4,8,6,8,8,6,6,5,4,2,1,3,2,3,1
5,7,5,8,8,8,8,7,5,1,2,3,2,3,1
6,7,8,7,5,5,5,6,7,2,1,3,2,3,1
7,6,5,7,7,6,5,6,5,2,1,3,1,3,2
8,7,5,8,7,5,7,4,4,2,1,3,2,3,1


## Step 2 (B) - Group (instead of Relabel)

Group values in B and C

In [ ]:
# Group responses into categories (to see if it improves interrater agreement)
# fixBC_group = group_categories(fixBC) 
fixBC_group = group_categories_overlap(fixBC) 
fixBC_group

,QB-F1-1,QB-F1-2,QB-F2-1,QB-F2-2,QC-F1-1,QC-F1-2,QC-F2-1,QC-F2-2
0,3,2,3,3,3,3,2,1
1,1,2,1,1,1,2,1,1
2,3,2,3,3,3,3,3,1
3,2,1,2,1,1,2,1,1
4,3,2,3,3,2,2,1,1
5,3,1,3,3,3,3,3,1
6,3,3,3,1,1,1,2,3
7,2,1,3,3,2,1,2,1
8,3,1,3,3,1,3,1,1
9,1,2,2,2,1,3,1,1


Recombine fixBC and fixD - GROUPED

In [ ]:
fixBC_group.index.name = "Index"
fixD.index.name = "Index"

In [ ]:
fix_group = fixBC_group.merge(fixD, on="Index")

df = fix_group

df

,QB-F1-1,QB-F1-2,QB-F2-1,QB-F2-2,QC-F1-1,QC-F1-2,QC-F2-1,QC-F2-2,QD-F1_1,QD-F1_2,QD-F1_3,QD-F2_1,QD-F2_2,QD-F2_3
Index,,,,,,,,,,,,,,
0,6,5,6,6,6,6,5,4,1,2,3,2,3,1
1,4,5,4,4,4,5,4,4,2,1,3,2,3,1
2,6,5,6,6,6,6,6,4,2,3,1,2,1,3
3,5,4,5,4,4,5,4,4,1,2,3,1,3,2
4,6,5,6,6,5,5,4,4,2,1,3,2,3,1
5,6,4,6,6,6,6,6,4,1,2,3,2,3,1
6,6,6,6,4,4,4,5,6,2,1,3,2,3,1
7,5,4,6,6,5,4,5,4,2,1,3,1,3,2
8,6,4,6,6,4,6,4,4,2,1,3,2,3,1


## Step 3 - Remove Zeroes

Remove columns with zeroes

In [ ]:
def drop_all_zeroes(df):
  df = df.loc[(df != 0).any(axis=1)]
  return df

In [ ]:
def drop_any_zeroes(df):
  df = df.loc[(df != 0).all(axis=1)]
  return df

In [ ]:
df = fixBC

In [ ]:
#drop any zeroes
df_noZeroes = drop_any_zeroes(df)

## Step 4 - Convert from Pandas Dataframe to a list (necessary for library)

In [ ]:
# Version without zeroes
df_noZeroes_list = df_noZeroes.values.tolist()

In [ ]:
# Version with zeroes
df_list = df.values.tolist()

# Inter rater agreement using Krippendorff's Alpha Score

In [ ]:
#https://en.wikipedia.org/wiki/Krippendorff%27s_alpha
#https://pypi.org/project/krippendorff/

!pip install krippendorff
import krippendorff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
res = krippendorff.alpha(df_noZeroes_list)

res

0.18671781086635386

In [ ]:
# Trying out krippendorff on data split into the two sections
df = df_filt
fixBC = fixed_q_BC(df)
fixD = fixed_q_D(df)

# fixBC = group_categories(fixBC) 

fixBC = drop_any_zeroes(fixBC)
fixD = drop_any_zeroes(fixD)

rate = fixBC.values.tolist()
rank = fixD.values.tolist()

res1 = krippendorff.alpha(rate)
print(res1)
res2 = krippendorff.alpha(rank)
print(res2)

0.22193537322801138
0.613775654819751


#### Try Fleiss kappa (couldnt get it to work)


In [ ]:
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters

In [ ]:
# Trying out fleiss on data split into the two sections
df = df_filt
fixBC = fixed_q_BC(df)
fixD = fixed_q_D(df)

# fixBC = group_categories(fixBC) 

fixBC = drop_any_zeroes(fixBC)
fixD = drop_any_zeroes(fixD)

rate = aggregate_raters(fixBC)
rank = aggregate_raters(fixD)

res1 = fleiss_kappa(rate)
print(res1)
res2 = fleiss_kappa(rank)
print(res2)

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/inter_rater.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  table = 1.0 * np.asarray(table)   #avoid integer division


ValueError: ignored

# Dictionary of all answered questions for each question


In [ ]:
def quest_dict(df):
  quest = dict()
  for col in df.columns:
    ans = list(df[col])
    ans = [val for val in ans if val != 0]
    quest[col] = ans
  return quest 

# Probably also make it 

In [ ]:
quest = quest_dict(df_all)

print(quest)

{'QB-F1-1': [4, 2, 4, 3, 4, 5, 4, 4, 3, 4, 2, 2, 4, 5, 3, 2, 5, 2, 5, 2, 4, 3, 5, 5, 5, 2, 3, 2, 5, 4, 4, 5, 5, 2, 4, 3, 2, 5, 5, 3, 2, 4, 3, 3, 4, 2, 5, 4, 4, 5, 5, 3, 5, 4, 3, 3, 4, 4, 4, 5, 2, 5, 4, 4, 5, 4], 'QB-F1-2': [3, 3, 3, 2, 3, 3, 2, 5, 2, 2, 3, 3, 4, 5, 4, 3, 4, 3, 5, 3, 5, 4, 3, 5, 2, 3, 2, 2, 2, 2, 4, 5, 5, 5, 4, 4, 3, 5, 3, 3, 4, 3, 2, 3, 4, 4, 2, 4, 4, 5, 5, 3, 5, 4, 3, 2, 3, 3, 2, 2, 5, 3, 3, 5, 3, 3], 'QB-F2-1': [5, 1, 4, 3, 4, 5, 5, 4, 4, 5, 3, 2, 5, 4, 4, 2, 5, 5, 5, 3, 4, 5, 5, 5, 5, 2, 4, 2, 5, 4, 4, 3, 5, 4, 5, 4, 2, 5, 5, 1, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 4, 4, 5, 5, 4, 4, 5, 5, 3, 5, 3, 5, 4, 5, 5, 5, 4], 'QB-F2-2': [4, 1, 4, 1, 5, 5, 5, 2, 4, 4, 3, 1, 1, 5, 4, 2, 2, 2, 2, 4, 2, 5, 5, 5, 5, 2, 4, 2, 5, 4, 3, 3, 4, 3, 5, 2, 2, 5, 1, 5, 5, 5, 3, 4, 2, 2, 1, 3, 5, 4, 5, 5, 5, 4, 5, 2, 3, 2, 5, 2, 5, 3, 5, 5, 5, 3], 'QB-R1-1': [5, 4, 4, 2, 2, 1, 3, 4, 2, 5, 4, 4, 5, 5, 5, 5, 4, 5], 'QB-R1-2': [2, 3, 2, 3, 3, 1, 2, 2, 1, 3, 2, 2, 3, 1, 1, 2, 4, 1], 'QB-R2-1': [5, 5, 

In [ ]:
print(df_all.columns)
print(len(df_all.columns))

Index(['QB-F1-1', 'QB-F1-2', 'QB-F2-1', 'QB-F2-2', 'QB-R1-1', 'QB-R1-2',
       'QB-R2-1', 'QB-R2-2', 'QB-R3-1', 'QB-R3-2', 'QB-R4-1', 'QB-R4-2',
       'QB-R5-1', 'QB-R5-2', 'QB-R6-1', 'QB-R6-2', 'QB-R7-1', 'QB-R7-2',
       'QB-R8-1', 'QB-R8-2', 'QB-R9-1', 'QB-R9-2', 'QB-R10-1', 'QB-R10-2',
       'QB-R11-1', 'QB-R11-2', 'QB-R12-1', 'QB-R12-2', 'QC-F1-1', 'QC-F1-2',
       'QC-F2-1', 'QC-F2-2', 'QC-R1-1', 'QC-R1-2', 'QC-R2-1', 'QC-R2-2',
       'QC-R3-1', 'QC-R3-2', 'QC-R4-1', 'QC-R4-2', 'QC-R5-1', 'QC-R5-1.1',
       'QC-R6-1', 'QC-R6-2', 'QC-R7-1', 'QC-R7-2', 'QC-R8-1', 'QC-R8-2',
       'QC-R9-1', 'QC-R9-2', 'QC-R10-1', 'QC-R10-2', 'QC-R11-1', 'QC-R11-2',
       'QC-R12-1', 'QC-R12-2', 'QD-F1_1', 'QD-F1_2', 'QD-F1_3', 'QD-F2_1',
       'QD-F2_2', 'QD-F2_3', 'QD-R1_1', 'QD-R1_2', 'QD-R1_3', 'QD-R2_1',
       'QD-R2_2', 'QD-R2_3', 'QD-R3_1', 'QD-R3_2', 'QD-R3_3', 'QD-R4_1',
       'QD-R4_2', 'QD-R4_3', 'QD-R5_1', 'QD-R5_2', 'QD-R5_3', 'QD-R6_1',
       'QD-R6_2', 'QD-R6_3', 'QD-R7_1

In [ ]:
# Columns to take for each section
cols = df_all.columns

e2e_fluent_col = cols[0:27:2]
e2e_meaning_col = cols[1:28:2]
pipe_fluent_col = cols[28:55:2]
pipe_meaning_col = cols[29:56:2]

base_comp_col = [cols[i] for i in [57,61,64,65,68,73,75,79,81,85,88,89]]
e2e_comp_col = [cols[i] for i in [58,59,63,66,70,71,76,78,80,84,87,90]]
pipe_comp_col = [cols[i] for i in [56,60,62,67,69,72,74,77,82,83,86,91]]

In [ ]:
# Make dataframe for each section
def dict_from_cols(cols):
  dic = dict()
  for col in cols:
    dic[col] = quest[col]
  return dic

In [ ]:
# Arrays for each section
e2e_fluent = dict_from_cols(e2e_fluent_col)
e2e_meaning = dict_from_cols(e2e_meaning_col)
pipe_fluent = dict_from_cols(pipe_fluent_col)
pipe_meaning = dict_from_cols(pipe_meaning_col)

base_comp = dict_from_cols(base_comp_col)
e2e_comp = dict_from_cols(e2e_comp_col)
pipe_comp = dict_from_cols(pipe_comp_col)